# Car Price Predictor
This project takes data from multiple CSV files, taken from 100,000 UK used car listings, and produces a linear regression model to predict a cars price from its features, such as make, model, mileage, etc.